In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

import sqlite3
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

The code below was provided by Abhineet

In [2]:
from glob import glob

In [3]:
csv_files = glob("./zippedData/*.csv.gz")

In [4]:
d = {}
for file in csv_files:
    d[file] = pd.read_csv(file)

In [5]:
csv_files_dict = {}
for filename in csv_files:
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".", "_") # cleaning the filenames
    filename_df = pd.read_csv(filename, index_col=0)
    csv_files_dict[filename_cleaned] = filename_df

In [6]:
conn = sqlite3.connect("movies_db.sqlite")

In [7]:
def create_sql_table_from_df(df, name, conn):
    try:
        df.to_sql(name, conn)
        print(f"Created table {name}")
    
    except Exception as e:
        print(f"could not make table {name}")
        print(e)

In [8]:
for name, table in csv_files_dict.items():
    create_sql_table_from_df(table, name, conn)

could not make table bom_movie_gross_gz
Table 'bom_movie_gross_gz' already exists.
could not make table imdb_name_basics_gz
Table 'imdb_name_basics_gz' already exists.
could not make table imdb_title_akas_gz
Table 'imdb_title_akas_gz' already exists.
could not make table imdb_title_basics_gz
Table 'imdb_title_basics_gz' already exists.
could not make table imdb_title_crew_gz
Table 'imdb_title_crew_gz' already exists.
could not make table imdb_title_principals_gz
Table 'imdb_title_principals_gz' already exists.
could not make table imdb_title_ratings_gz
Table 'imdb_title_ratings_gz' already exists.
could not make table tmdb_movies_gz
Table 'tmdb_movies_gz' already exists.
could not make table tn_movie_budgets_gz
Table 'tn_movie_budgets_gz' already exists.


In [9]:
conn.execute("select name from sqlite_master where type='table';").fetchall()

[('bom_movie_gross_gz',),
 ('imdb_name_basics_gz',),
 ('imdb_title_akas_gz',),
 ('imdb_title_basics_gz',),
 ('imdb_title_crew_gz',),
 ('imdb_title_principals_gz',),
 ('imdb_title_ratings_gz',),
 ('tmdb_movies_gz',),
 ('tn_movie_budgets_gz',)]

Below, we start our own code

Since we will be working with Pandas we need to organize our DFs. 
Let's start by renaming all of them and adding them to a new dictionary. 
By creating a new dictionary of DFs, we can manipulate the data without messing up the originals in csv_files_dict.

In [10]:
df_repository = {} #by creating a dictionary, we can manipulate the data without messing up the originals in csv_files_dict
for filename in csv_files:
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".", "_") # cleaning the filenames    
    exec('df_'+ filename_cleaned + """ = csv_files_dict['""" + filename_cleaned + """']""") #executing code using strings
    exec("""df_repository[""" + """'""" + filename_cleaned + """']""" + """= df_""" + filename_cleaned )

As seen below, the dataframes contain duplicates, which should be removed

In [11]:
for db in df_repository.keys():
    print(db, 'contains ', df_repository[db].duplicated().sum(), ' duplicates')

bom_movie_gross_gz contains  11  duplicates
imdb_name_basics_gz contains  239  duplicates
imdb_title_akas_gz contains  2220  duplicates
imdb_title_basics_gz contains  122  duplicates
imdb_title_crew_gz contains  22915  duplicates
imdb_title_principals_gz contains  114187  duplicates
imdb_title_ratings_gz contains  45074  duplicates
tmdb_movies_gz contains  1020  duplicates
tn_movie_budgets_gz contains  0  duplicates


In [12]:
def clean_duplicates(dataframe): #returns a dataframe without duplicates
    cleaned_filter = dataframe.duplicated()
    index_to_drop = dataframe.loc[cleaned_filter].index
    return dataframe.drop(index=index_to_drop)

In [13]:
for db in df_repository.keys(): #loop to parse dataframes through the clean_duplicates() function we wrote above
    df_repository[db] = clean_duplicates(df_repository[db])

In [14]:
for db in df_repository.keys():
    print(db, 'contains ', df_repository[db].duplicated().sum(), ' duplicates') #checking if it worked

bom_movie_gross_gz contains  0  duplicates
imdb_name_basics_gz contains  0  duplicates
imdb_title_akas_gz contains  0  duplicates
imdb_title_basics_gz contains  0  duplicates
imdb_title_crew_gz contains  0  duplicates
imdb_title_principals_gz contains  0  duplicates
imdb_title_ratings_gz contains  0  duplicates
tmdb_movies_gz contains  0  duplicates
tn_movie_budgets_gz contains  0  duplicates


Elimination of NaN values

In [15]:
def NaN_to_zero(DataFrame, DataSeries, change_into=0):
    filt = DataSeries.isna()
    DataFrame.loc[filt] = change_into
    return DataFrame

In [16]:
#testing the NaN_to_zero function 
df_bom_movie_gross_gz.isna().sum()

studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [17]:
NaN_to_zero(df_bom_movie_gross_gz, df_bom_movie_gross_gz.foreign_gross).isna().sum()

studio             4
domestic_gross    28
foreign_gross      0
year               0
dtype: int64

Now that we know it works, we can start cleaning up NaN values. After looking through the DFs, we should find other numerical columns with NaN values, and then apply the function to them. 

Some columns that are supposed to be numerical actually have data stored as strings. Let's write a function that transforms the data

In [19]:
def string_to_float(string):
    if string == None:
        new_string = 0
    else:
        if (type(string) != int) and (type(string) != float):
            new_string = string.replace('$', '')
            new_string = new_string.replace(',', '')

        else:
            new_string = string
    return float(new_string)


def series_string_to_float(dataseries):
    new_dataseries = dataseries.apply(string_to_float)
    return new_dataseries


In [23]:
type(df_bom_movie_gross_gz.foreign_gross[0]) #the data here is stored as strings instead of numerical values

str

In [24]:
new_series = series_string_to_float(df_bom_movie_gross_gz.foreign_gross)
type(new_series[0]) #checking if it works

numpy.float64

After cleaning data, we can start creating new features

In [26]:
df_bom_movie_gross_gz.foreign_gross = series_string_to_float(df_bom_movie_gross_gz.foreign_gross)

In [28]:
df_bom_movie_gross_gz['total_gross'] = df_bom_movie_gross_gz['foreign_gross'].add(df_bom_movie_gross_gz['domestic_gross'], fill_value=0.0)
df_bom_movie_gross_gz.head()

,studio,domestic_gross,foreign_gross,year,total_gross
title,,,,,
Toy Story 3,BV,415000000.0,652000000.0,2010,1.067000e+09
Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,1.025500e+09
Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010,9.603000e+08
Inception,WB,292600000.0,535700000.0,2010,8.283000e+08
Shrek Forever After,P/DW,238700000.0,513900000.0,2010,7.526000e+08


In order to simplify the manipulation of data, the column names of each df could be standardized ('title' vs 'movie title' vs 'original title', etc)